In [16]:
import pandas as pd

df = pd.read_csv("homepage_data.csv")
df = df[~df['Personal Website'].str.endswith('.sg').astype(bool)]
df.head()

,Name,Personal Website,About Us Text
0,Albert Chi-Shing CHUNG,https://cse.hkust.edu.hk/admin/people/faculty/...,Albert Chung is a Full Professor at the Depart...
1,Andrew Wing-On POON,https://eeawpoon.people.ust.hk/,Professor Poon received his BA degree from the...
2,Antoni Bert CHAN,https://www.cs.cityu.edu.hk/~abchan/,Dr. Antoni Chan is a Professor at the City Uni...
3,Brian Kan-Wing MAK,https://cse.hkust.edu.hk/admin/people/faculty/...,Dr. Mak is currently an Associate Professor in...
4,CHAN Chee Yong,https://sgps.cuhk.edu.hk/profile/professor-cha...,Professor Chee Hon Chan is an Assistant Profes...


In [26]:
import pandas as pd
df = pd.read_csv("data/0721-month-analysis-data.csv")
# df.columns
df = df[df['Pre-2014 School'].isin(['CityU', 'CUHK', 'HKU', 'HKUST'])]
df['Researcher Cleaned Name'].unique().shape


(137,)

In [17]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import requests
from markdownify import markdownify as md
import json

load_dotenv(".env")

client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

def process_homepage_text(text: str):
    response = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": """Extract the following qualities, with justification in a JSON formatted output. 
Try to infer these from the way the author writes about themselves. Use and describe subtle language patterns to extract this understanding about a given individual.  JSON output format: 
{
    "friendliness-justification": "Explain your friendliness quote with citations",
    "friendliness": "Scale of 1-10, how much they exhibit traits of cameraderie, teamwork and collaboration. 10 being heavy-emphasis on friendliness.",
    "we-vs-i-justification": "Explain your we-vs-i quote with citations",
    "we-vs-i": "Scale of 1-10, how much they talk about themselves vs their collaborators. 10 being heavy-emphasis on we.",
    "risk-averse-justification": "Explain your risk-averse quote with citations",
    "risk-averse": "Scale of 1-10, how they feel about taking risks. 10 being very risk-averse.",
    "career-orientation-justification": "Explain your career-orientation quote with citations",
    "career-orientation": "How much a person cares about their career. 10 being very career-oriented.",
}"""
        }, {
            "role": "user",
            "content": text
        }],
        response_format={
            "type": "json_object"
        },
        model="gpt-4o"
    )
    response = json.loads(response.choices[0].message.content)
    return response


In [18]:
from rich import print
from tqdm import tqdm

# TODO: append university profile w/ personal website text
# TODO: make sure that results aren't correlated w/ more text (more personable because they wrote more) 

researcher_profile = []

for index, row in tqdm(df.iterrows()):
    text = row['Personal Website']
    response = process_homepage_text(text)
    response["name"] = row['Name']
    researcher_profile.append(response)
    print(response)
    

0it [00:00, ?it/s]

{
    'friendliness-justification': 'The webpage does not provide direct language or personal narrative indicating a 
sense of friendliness, as it is a professional faculty profile primarily focusing on academic and career 
achievements.',
    'friendliness': '5',
    'we-vs-i-justification': "In most academic profiles, the focus is predominantly on individual accomplishments 
like papers, research focus, and projects, suggesting a balanced emphasis on 'I' within a collaborative 'We' 
environment, but specific language from the profile is missing here.",
    'we-vs-i': '6',
    'risk-averse-justification': "The focus on stable academic positions and achievements in such profiles 
indicates a moderate degree of risk aversion, though specific language from the provided URL isn't available.",
    'risk-averse': '7',
    'career-orientation-justification': 'Given the nature of the URL (an academic faculty profile), a significant 
emphasis is placed on career achievements and positions, indicating a high career orientation even without specific
text available.',
    'career-orientation': '9',
    'name': 'Albert Chi-Shing CHUNG'
}

1it [00:04,  4.24s/it]

{
    'friendliness-justification': 'The document provided does not contain any written content that would allow for 
an inference on friendliness. It is a URL link.',
    'friendliness': '5',
    'we-vs-i-justification': 'The document provided does not contain any written content that would allow for an 
inference on the use of collective vs individualistic language. It is a URL link.',
    'we-vs-i': '5',
    'risk-averse-justification': 'The document provided does not contain any written content that would allow for 
an inference on risk tolerance. It is a URL link.',
    'risk-averse': '5',
    'career-orientation-justification': 'The document provided does not contain any written content that would 
allow for an inference on career orientation. It is a URL link.',
    'career-orientation': '5',
    'name': 'Andrew Wing-On POON'
}

2it [00:08,  4.10s/it]

...

{
    'friendliness-justification': 'The personal webpage does not provide explicit text or statements from which to 
directly gauge friendliness. Typically, an absence of personal anecdotes, testimonials, or informal language can 
imply a neutral to low emphasis on personal warmth.',
    'friendliness': '5',
    'we-vs-i-justification': 'Professional academic profiles generally focus on ‘I’ when detailing personal 
achievements such as publications or research. Without specific collaborative language or examples, it is difficult
to assess this attribute in detail.',
    'we-vs-i': '4',
    'risk-averse-justification': 'Since this is more a professional academic profile, there is no explicit mention 
of risk-taking or aversion. Academics typically show cautious progress marked by peer-reviewed publications, which 
may hint at a balanced approach.',
    'risk-averse': '5',
    'career-orientation-justification': "The detailed professional experience, complete listing of publications, 
and academic achievements suggest a strong career focus. Presenting one's work in such a manner typically indicates
a high career orientation.",
    'career-orientation': '9',
    'name': 'CHUA Tat Seng'
}

6it [00:30,  5.37s/it]

{
    'friendliness-justification': 'Since there are no personal statements from the individual, it is not possible 
to determine friendliness from the provided link.',
    'friendliness': 'N/A',
    'we-vs-i-justification': "Without a personal statement or bio to analyze, it is not possible to assess the 
emphasis on 'we' vs 'I' from the provided link.",
    'we-vs-i': 'N/A',
    'risk-averse-justification': "There is no text provided that indicates the individual's approach to risk from 
the provided link.",
    'risk-averse': 'N/A',
    'career-orientation-justification': "The linked profile likely showcases professional achievements and 
publications, which generally indicate a focus on career. However, without specific statements from the individual,
it's hard to assess the personal level of emphasis.",
    'career-orientation': 'N/A',
    'name': 'Cham Tat Jen'
}

7it [00:37,  5.43s/it]


KeyboardInterrupt: 

In [ ]:
with open("homepage_data_analyzed.json", "w+") as f:
    json.dump(researcher_profile, f)

In [8]:
# TODO: save to 0721-month-analysis-data, save to new file